## IPVisibility RPM

In [ ]:
from IPEnvironmentShapeRobot import CollisionCheckerShapeRobot
from IPEnvironmentShapeRobot import ShapeRobot, ShapeRobotWithOrientation
from shapely.geometry import Point, Polygon, LineString
from shapely.affinity import scale, translate
import matplotlib.pyplot as plt

import numpy as np



# Definition of a testing environment
obs1 = LineString([(5,18),(5,8),(20,8),(20,18)]).buffer(1.0)
obs2 = LineString([(15,15), (15,25), (30,25), (30,10)]).buffer(1.0)

# obs2 = scale(obs1, yfact=-1, origin='center')
# obs2 = translate(obs2, xoff=10, yoff=7)
trapField = dict()
trapField["obs1"] = obs1
trapField["obs2"] = obs2

# define robot geometry
robot_shape = Polygon([
    (-0.5, -0.5),  # unten links
    (3, -0.5),     # unten rechts
    (3, 0.0),      # Innenknick rechts
    (0.5, 0.0),    # Innenknick unten horizontal
    (0.5, 1.0),    # oben links vom L-Schenkel
    (-0.5, 1.0)    # oben links außen
])
shape_robot = ShapeRobotWithOrientation(robot_shape, limits=[[0.0, 22.0,], [0.0, 22.0], [-np.pi, np.pi]])

environment = CollisionCheckerShapeRobot(trapField, shape_robot)

In [ ]:
# define robot geometry

fig_local = plt.figure(figsize=(10,10))
ax = fig_local.add_subplot(1,1,1)
environment.drawObstacles(ax)

In [ ]:
from IPVisibilityPRM import VisPRM 

visPRM = VisPRM(environment)
visConfig = dict()
visConfig["radius"] = 10
visConfig["ntry"] = 50

start = [[25,5,0]]
goal  = [[15,12,0]]

solution = visPRM.planPath(start, goal, visConfig)

if len(solution) == 0:
    print("No solution found")

solution

In [ ]:
from IPVISVisibilityPRM import visibilityPRMVisualize
fig_local = plt.figure(figsize=(7,7))
ax = fig_local.add_subplot(1,1,1)
visibilityPRMVisualize(visPRM,solution,ax=ax)

In [ ]:
import copy
def interpolate_line(startPos, endPos, step_l):
    steps = []
    line = np.array(endPos) - np.array(startPos)
    line_l = np.linalg.norm(line)
    step = line / line_l * step_l
    n_steps = np.floor(line_l / step_l).astype(np.int32)
    c_step = np.array(startPos, dtype=np.float32)
    for i in range(n_steps):
        steps.append(copy.deepcopy(c_step))
        c_step += step
    if not (c_step == np.array(endPos)).all():
        steps.append(np.array(endPos))
    return steps


In [ ]:
# from IPVISVisibilityPRM import visibilityPRMVisualizeWspace
# import matplotlib.animation
# matplotlib.rcParams['animation.embed_limit'] = 256
# from IPython.display import HTML, display

# planner=visPRM

# fig_local = plt.figure(figsize=(7, 7))
# ax = fig_local.add_subplot(1, 1, 1)
# ## get positions for solution
# solution_pos = [planner.graph.nodes[node]['pos'] for node in solution]
# ## interpolate to obtain a smoother movement
# i_solution_pos = [solution_pos[0]]
# for i in range(1, len(solution_pos)):
#     segment_s = solution_pos[i-1]
#     segment_e = solution_pos[i]
#     i_solution_pos = i_solution_pos + interpolate_line(segment_s, segment_e, 0.5)[1:]
# ## animate
# frames = len(i_solution_pos)

# r = environment.robot
# workSpaceLimits = environment.robot.getLimits()

# def animate(t):
#     ## clear taks space figure
#     ax.cla()
#     ## fix figure size
#     ax.set_xlim(workSpaceLimits[0])
#     ax.set_ylim(workSpaceLimits[1])
#     ## draw obstacles
#     #environment.drawObstacles(ax)
#     ## update robot position
#     pos = i_solution_pos[t]
#     r.setTo(pos)
#     visibilityPRMVisualizeWspace(visPRM,solution,ax=ax)
    

# ani = matplotlib.animation.FuncAnimation(fig_local, animate, frames=frames)
# html = HTML(ani.to_jshtml())
# display(html)
# plt.close()



## Smooth the path

### Generate random points on edges

In [ ]:
import networkx as nx
from PathSmoother import random_pt_on_edge


knot_positions = nx.get_node_attributes(visPRM.graph,'pos')
random_edge_pts = []

for i in range(len(solution) - 1):
    u = solution[i]
    v = solution[i+1]
    
    random_edge_pts.append(random_pt_on_edge(knot_positions[u], knot_positions[v]))

random_edge_pts

### Check for obstacles between those points 

In [ ]:
from PathSmoother import findRandomShortcut, insertAndConnectPointsOnEdges


print("Solution before: ", solution)
id_counter = 0
color = "#e28a0e"

u, v = findRandomShortcut(visPRM, random_edge_pts)
print("u: ", u, " v: ", v)
insertAndConnectPointsOnEdges(visPRM.graph, solution, random_edge_pts, u, v, id_counter, color)
print("Solution after: ", solution)


In [ ]:
from IPVISVisibilityPRM import visibilityPRMVisualize
opt_fig_local = plt.figure(figsize=(7,7))
ax = opt_fig_local.add_subplot(1,1,1)
visibilityPRMVisualize(visPRM, solution, ax=ax)

# print(list(visPRM.graph.edges("S1")))
# print(list(visPRM.graph.edges("S2")))
# print(visPRM.graph.nodes[23]["pos"])